In [1]:
using JuMP, Gurobi, CSV, StatsBase, DataFrames, Plots, Dates, Suppressor, DelimitedFiles

### Optimal Dock Allocation
Given what you think the next month's demand is going to be, what should the dock sizes look like?

Complications: 
1. change as little stations as possible to lower some fixed costs
2. station size should be more or less similar if the two neighbourhood's populations are similar


In [2]:
bikes = CSV.read("status_station_weather.csv", header = true, missingstring = "NA")
bikes_august = bikes[bikes[!, :date] .>= Date("2015-08-01"), 3:end]
size(bikes_august)

(2170, 32)

In [3]:
m = reshape(bikes_august[!, :avg_docks_available], (31, 70))

31×70 reshape(::Array{Union{Missing, Float64},1}, 31, 70) with eltype Union{Missing, Float64}:
 16.5493  2.29722  3.93194  13.034    …  11.0111    8.05208  10.4319 
 17.4241  3.17052  4.57587  13.1987       9.3013    8.07587   9.0    
 16.984   4.47986  7.06389  13.4576       7.91806   8.07292  10.459  
 13.9806  5.34931  4.60278  13.0458      11.15      8.42847  10.766  
 18.9951  4.47847  3.3125   12.8146      10.8493    8.04514   9.45625
 17.3236  4.73333  5.79792  12.6868   …   9.85833   8.03125   9.95486
 17.1313  4.03819  7.32847  11.6479       8.55625   7.76528  11.459  
 16.0868  3.54583  7.85556  12.0          9.86528   7.05556  11.5333 
 17.241   3.93819  9.75139  12.0          7.22361   7.15833  10.6194 
 17.4347  4.5      7.1      11.9778       8.75694   6.82292   7.93958
 20.3007  4.91875  5.23472  12.909    …   9.55278   6.58681   7.06111
 17.4208  7.41875  7.24653  14.3792       7.91111   7.48194   6.19583
 18.9483  6.72714  5.98157  14.9043       7.74841   8.00071   4.8

In [4]:
prediction = CSV.read("prediction_cleaned.csv", header = true, missingstring = "NA")
m_predicted = convert(Matrix, prediction)

31×70 Array{Float64,2}:
 17.7591  2.97928  5.20994  11.6826  …  12.0459  8.8187   8.25105  9.76565
 16.5297  3.92793  5.82223  11.6826     12.0459  9.47717  8.25105  9.0949 
 15.5534  4.4989   5.82223  11.6826     12.0459  8.8187   8.25105  9.0949 
 14.2173  5.20994  5.82223  11.6826     12.0459  8.8187   8.25105  9.0949 
 13.1882  5.82223  5.82223  11.6826     11.6826  8.19681  8.25105  9.0949 
 12.6806  5.82223  5.82223  11.6826  …  11.6826  8.8187   8.25105  9.0949 
 13.4586  5.82223  5.82223  11.6826     11.6826  8.8187   8.25105  9.52642
 14.2435  5.82223  5.82223  11.6826     11.6826  8.8187   8.25105  9.52642
 14.2435  5.82223  5.82223  11.6826     11.6826  8.8187   8.25105  9.0949 
 13.1882  5.82223  5.82223  11.6826     11.6826  8.8187   8.25105  9.0949 
 12.6806  5.82223  5.82223  11.6826  …  11.6826  8.8187   8.25105  9.0949 
 11.6826  5.82223  5.82223  11.6826     11.6826  8.8187   8.25105  9.0949 
 11.6826  5.82223  5.82223  11.6826     11.6826  8.8187   8.25105  9.0949 
 

In [5]:
function optimizeDocks(m, min_docks, λ, M)
        model = Model(solver=GurobiSolver(OutputFlag=0, TimeLimit=60, InfUnbdInfo=1))

        @variable(model, t)
        @variable(model, y[1:31, 1:70], Bin)  #1 if there less than 4 bikes at the station
        @variable(model, x[1:70], Int)        #docks moved from station i
        @variable(model, z[1:70], Bin)        #1 if station i's dock size is changed

        @objective(model, Min, t + λ * sum(z))

        #cannot take more bikes than there are
        @constraint(model, [i = 1:31, j = 1:70], x[j] >= - m[i, j])
    
        #using only the docks within the system
        @constraint(model, sum(x) == 0)

        @constraint(model, t .>= sum(y, dims =2))

        # y[i, j] = 1 when m[i, j] + x[i] < 4, which means on the jth day and ith location, 
        # there are less than 4 docks available 
        @constraint(model, [i = 1:31, j = 1:70], m[i, j] + x[i] >= min_docks - M * y[i, j])
        @constraint(model, [i = 1:31, j = 1:70], m[i, j] + x[i] <= (min_docks - 0.001) + M * (1 - y[i, j]))


        #sparsity constraint: touch as little stations as possible
        @constraint(model, x .<= M*z)
        @constraint(model, x .>= -M*z)

        solve(model)
    return getobjectivevalue(model), getvalue(x), getvalue(z), getvalue(y)
end

optimizeDocks (generic function with 1 method)

Testing the model on the true data. 

In [6]:
opt_objective_true, opt_x_true, opt_z_true, opt_y_true = optimizeDocks(m, 5, 0.5, 100);
println("Optimal objective value is ", opt_objective_true)
println("Bikes moved: ", sum(abs.(opt_x_true)))
println("Stations changed: ", sum(opt_z_true))

Academic license - for non-commercial use only
Optimal objective value is 9.5
Bikes moved: 4.0
Stations changed: 3.0


Testing the model on the predictions from Optimal Regression Tree. 

In [7]:
opt_objective_pred, opt_x_pred, opt_z_pred, opt_y_pred = optimizeDocks(m_predicted, 5, 0.5, 100);
println("Optimal objective value is ", opt_objective_pred)
println("Bikes moved: ", sum(abs.(opt_x_pred)))
println("Stations changed: ", sum(opt_z_pred))

Academic license - for non-commercial use only
Optimal objective value is 2.0
Bikes moved: 12.0
Stations changed: 4.0


### Introducing the fairness consideration

In [8]:
station_in_zipcode = CSV.read("station_in_zipcode.csv", header = true, missingstring = "NA")
station_in_zipcode = station_in_zipcode[!, 4:end]
station_in_zipcode = convert(Matrix, station_in_zipcode)

population_per_zipcode = CSV.read("population_per_zipcode.csv", header = true, missingstring = "NA")
population_per_zipcode = convert(Array, population_per_zipcode[!, 2]);

In [9]:
function optimizeDocks(m, min_docks, λ, β, M)
        model = Model(solver=GurobiSolver(OutputFlag=0, TimeLimit=60, InfUnbdInfo=1))

        @variable(model, t)
        @variable(model, y[1:31, 1:70], Bin)  #1 if there less than 4 bikes at the station
        @variable(model, x[1:70], Int)        #docks moved from station i
        @variable(model, z[1:70], Bin)        #1 if station i's dock size is changed
        @variable(model, ratio[1:5, 1:31])    #ratio of docks available per person per zipcode per day
        @variable(model, L)

        @objective(model, Min, t + λ * sum(z) + β * L)

        #cannot take more bikes than there are
        @constraint(model, [i = 1:31, j = 1:70], x[j] >= - m[i, j])
    
        #using only the docks within the system
        @constraint(model, sum(x) == 0)

        @constraint(model, t .>= sum(y, dims =2))

        # y[i, j] = 1 when m[i, j] + x[i] < 4, which means on the jth day and ith location, 
        # there are less than 4 docks available 
        @constraint(model, [i = 1:31, j = 1:70], m[i, j] + x[i] >= min_docks - M * y[i, j])
        @constraint(model, [i = 1:31, j = 1:70], m[i, j] + x[i] <= (min_docks - 0.001) + M * (1 - y[i, j]))

        #sparsity constraint: touch as little stations as possible
        @constraint(model, x .<= M*z)
        @constraint(model, x .>= -M*z)
    
        #fairness
        @constraint(model, [i = 1:5, j = 1:31], ratio[i, j] == ((m[j, :] + x)' * station_in_zipcode[:, i]) / population_per_zipcode[i])
        @constraint(model, [i = 1:5, j = 1:31, k = 1:5, h = 1:31], ratio[i, j] - ratio[k, h] <= L)
        @constraint(model, [i = 1:5, j = 1:31, k = 1:5, h = 1:31], ratio[i, j] - ratio[k, h] >= -L)    
    
        solve(model)
    return getobjectivevalue(model), getvalue(x), getvalue(z), getvalue(y), getvalue(L), getvalue(ratio)
end

optimizeDocks (generic function with 2 methods)

In [10]:
opt_objective_true_2, opt_x_true_2, opt_z_true_2, opt_y_true_2, opt_L_true_2, opt_ratio_true_2 = optimizeDocks(m, 5, 0.5, 0.5, 100);
println("Optimal objective value is ", opt_objective_true_2)
println("Bikes moved: ", sum(abs.(opt_x_true_2)))
println("Stations changed: ", sum(opt_z_true_2))

Academic license - for non-commercial use only
Optimal objective value is 9.57779653875306
Bikes moved: 10.0
Stations changed: 3.0


In [11]:
opt_objective_pred_2, opt_x_pred_2, opt_z_pred_2, opt_y_pred_2, opt_L_pred_2, opt_ratio_pred_2 = optimizeDocks(m_predicted, 5, 0.5, 0.5, 100);
println("Optimal objective value is ", opt_objective_pred_2)
println("Bikes moved: ", sum(abs.(opt_x_pred_2)))
println("Stations changed: ", sum(opt_z_pred_2))

Academic license - for non-commercial use only
Optimal objective value is 2.077510366034596
Bikes moved: 12.0
Stations changed: 4.0


In [12]:
cat(opt_x_true, opt_x_true_2, opt_x_pred, opt_x_pred_2, dims = 2)

70×4 Array{Float64,2}:
 0.0   0.0   3.0   3.0
 0.0   0.0   2.0   2.0
 0.0   0.0   1.0   1.0
 0.0   0.0  -0.0   0.0
 0.0   0.0  -0.0   0.0
 0.0   0.0  -0.0   0.0
 0.0   0.0  -0.0   0.0
 0.0  -0.0  -0.0   0.0
 0.0   0.0  -0.0   0.0
 0.0   0.0  -0.0   0.0
 0.0   0.0  -0.0   0.0
 0.0   0.0  -0.0   0.0
 0.0   0.0  -0.0   0.0
 ⋮                    
 0.0   0.0   0.0   0.0
 0.0   0.0   0.0   0.0
 0.0   0.0   0.0   0.0
 0.0   0.0   0.0   0.0
 0.0   0.0   0.0   0.0
 0.0   0.0   0.0   0.0
 0.0   0.0   0.0   0.0
 0.0   0.0   0.0   0.0
 0.0  -5.0   0.0  -6.0
 0.0   0.0   0.0   0.0
 0.0   0.0   0.0   0.0
 0.0   0.0   0.0   0.0

In [13]:
cat(opt_z_true, opt_z_true_2, opt_z_pred, opt_z_pred_2, dims = 2)

70×4 Array{Float64,2}:
 0.0   0.0   1.0   1.0
 0.0   0.0   1.0   1.0
 0.0   0.0   1.0   1.0
 0.0  -0.0  -0.0   0.0
 0.0   0.0  -0.0   0.0
 0.0   0.0  -0.0   0.0
 0.0   0.0  -0.0   0.0
 0.0   0.0  -0.0   0.0
 0.0  -0.0  -0.0   0.0
 0.0   0.0  -0.0   0.0
 0.0   0.0  -0.0   0.0
 0.0  -0.0  -0.0   0.0
 0.0   0.0  -0.0   0.0
 ⋮                    
 0.0  -0.0  -0.0  -0.0
 0.0  -0.0  -0.0  -0.0
 0.0  -0.0  -0.0  -0.0
 0.0  -0.0  -0.0  -0.0
 0.0  -0.0  -0.0  -0.0
 0.0  -0.0  -0.0  -0.0
 0.0  -0.0  -0.0  -0.0
 0.0  -0.0  -0.0  -0.0
 0.0   1.0  -0.0   1.0
 0.0  -0.0  -0.0  -0.0
 0.0  -0.0  -0.0  -0.0
 0.0  -0.0  -0.0  -0.0

In [14]:
println(sum(abs.(opt_x_true)))
println(sum(abs.(opt_x_true_2)))
println(sum(abs.(opt_x_pred)))
println(sum(abs.(opt_x_pred_2)))

4.0
10.0
12.0
12.0


Varying the min_docks parameter to get the total number of docks transported and stations touched. Other parameters remain constant: λ = 0.5, β = 0.5 and M = 100. 

In [43]:
list_changing_min_docks = []
for min_docks = 1:15
    x = @suppress begin optimizeDocks(m, min_docks, 0.5, 0.5, 100) end
    total_docks = sum(abs.(x[2]))
    stations_touched = sum(x[3])
    list_changing_min_docks = cat(list_changing_min_docks, min_docks, total_docks, stations_touched, dims = 1)
end
list_changing_min_docks = reshape(list_changing_min_docks, (3, length(1:15)))'

15×3 LinearAlgebra.Adjoint{Any,Array{Any,2}}:
  1    0.0   0.0
  2    0.0   0.0
  3   10.0   2.0
  4   10.0   5.0
  5   10.0   3.0
  6   10.0   3.0
  7   10.0   3.0
  8  234.0  47.0
  9  296.0  53.0
 10  358.0  60.0
 11  376.0  62.0
 12  402.0  65.0
 13  410.0  66.0
 14  430.0  70.0
 15  426.0  69.0

Varying the sparsity parameter. Other parameters remain constant: min_docks = 6, β = 0.5 and M = 100. 

In [38]:
list_changing_lambda = []
for λ = collect(0.0:0.1:2.0)
    x = @suppress begin optimizeDocks(m, 6, λ, 0.5, 100) end
    total_docks = sum(abs.(x[2]))
    stations_touched = sum(x[3])
    list_changing_lambda = cat(list_changing_lambda, λ, total_docks, stations_touched, dims = 1)
end
list_changing_lambda = reshape(list_changing_lambda, (3, length(collect(0.0:0.1:2.0))))'

21×3 LinearAlgebra.Adjoint{Any,Array{Any,2}}:
 0.0  410.0  70.0
 0.1  208.0  45.0
 0.2  208.0  45.0
 0.3  208.0  45.0
 0.4   10.0   3.0
 0.5   10.0   3.0
 0.6   10.0   3.0
 0.7   10.0   3.0
 0.8   10.0   3.0
 0.9   10.0   3.0
 1.0   10.0   3.0
 1.1   10.0   3.0
 1.2   10.0   3.0
 1.3   10.0   3.0
 1.4   10.0   3.0
 1.5   10.0   3.0
 1.6   10.0   3.0
 1.7    0.0   0.0
 1.8    0.0   0.0
 1.9    0.0   0.0
 2.0    0.0   0.0

With and without the fairness consideration β, the allocation changes. Other parameters remain constant: min_docks = 6, λ = 0.5 and M = 100. 

In [57]:
β = 0
x = optimizeDocks(m, 8, 0.5, β, 100)
writedlm("dock_allocation_NO_fairness.csv",  x[2], ',')

In [17]:
β = 5.0
x = optimizeDocks(m, 8, 0.5, β, 100)
opt_x = x[2]
writedlm("dock_allocation_WITH_fairness.csv",  x[2], ',')

Academic license - for non-commercial use only


In [59]:
writedlm("list_changing_min_docks.csv",  list_changing_min_docks, ',')
writedlm("list_changing_lambda.csv",  list_changing_lambda, ',')